1D cubic spline coefficients:
=============================

In [1]:
from itertools import product
import numpy as np

Let $f:\mathbb{Z} \to \mathbb{R}$ be a function we want to fit.

A cubic spline between data points $x_1 \in \mathbb{Z}$ and $x_2 = x_1 + 1$ is a cubic polynomial that somewhat agrees with $f$. It is most easy to work on the interval $[0, 1]$ by shifting, so instead of working with polynomials which are a function of $x$, we write them in terms of $x-x_1$. We represent this in abstractly as `xmx1`. We  also include `ymy1` and `zmz1` for the higher dimensions later.

The coefficients of this polynomial can be found by solving linear equations that put constraints on our polynomial. A popular choice is the Catmull-Rom spline, which demands that the polynomial agrees with $f$ in $x_1$ and $x_2$, and that the derivative of the polynomial in those points is equal to the central differences derivative of $f$ in the same points. As a result, the coefficients are linear combinations of the function values in the 4 points surrounding the interval $[x_1, x_2]$. We will represent these function values abstracly as `f0, f1, f2, f3`

So the polynomial of `xmx1` we are looking for has coefficients that are linear combinations of `f0, f1, f2, f3`. This is the same as a linear combination of `f0, f1, f2, f3` with coefficients that are polynomials of `xmx1`. We will work with this representation from the beginning.

In [2]:
# the coefficients which are polynomials of xmx1 are in this ring
S.<xmx1, ymy1, zmz1> = PolynomialRing(QQ)

# the linear combination of f0, f1, f2, f3 we are looking for
# is in this ring (whith coefficients in the previous ring), with homogenous degree one
R.<f0,f1,f2,f3> = PolynomialRing(S)

# the central differences in x1 and x2
df1 = (f2 - f0)/2
df2 = (f3 - f1)/2

We look for the unknown polynomial of the form

$p(x-x_1) = c_0 (x-x_1)^3 - c_1 (x-x_1)^2 + c_2 (x-x_1) + c_3$

on which we demand

 - $p(0) = f_1$
 - $p(1) = f_2$
 - $p'(0) = (f_2 - f_0)/2$
 - $p'(1) = (f_3 - f_1)/2$

In [3]:
# this system represents the constraints above
A = Matrix(R, [[0, 0, 0, 1],
               [1, 1, 1, 1],
               [0, 0, 1, 0],
               [3, 2, 1, 0]])
b = vector(R, [f1, f2, df1, df2])

pretty_print(A, LatexExpr("c ="), Matrix(b).T)

[0 0 0 1]
[1 1 1 1]
[0 0 1 0]
[3 2 1 0] c = [                f1]
[                f2]
[(-1/2)*f0 + 1/2*f2]
[(-1/2)*f1 + 1/2*f3]

In [4]:
# we solve this system to get the unknown coefficients
c = A \ b

# the coefficients of c are now in the fraction field of S, but the denominators are all 1
# indeed:
print(list(map(lambda x: x.denominator(), c)))

[1, 1, 1, 1]


In [5]:
# so we can just take the numerators
c = list(map(lambda x: x.numerator(), c))

# this is the polynomial we where looking for
p = c[0]*(xmx1)^3 + c[1] * (xmx1)^2 + c[2]*(xmx1) + c[3]

# and because of the used ring structure, sage automatically writes this as a linear
# combination of f0,f1,f2,f3 with coefficients that are polynomials of xmx1
print(p)

(-1/2*xmx1^3 + xmx1^2 - 1/2*xmx1)*f0 + (3/2*xmx1^3 - 5/2*xmx1^2 + 1)*f1 + (-3/2*xmx1^3 + 2*xmx1^2 + 1/2*xmx1)*f2 + (1/2*xmx1^3 - 1/2*xmx1^2)*f3


2D cubic spline coefficients
============================

The same thing can be done in 2D. There are now 16 function values that determine the spline.

In [6]:
R2 = PolynomialRing(S, ['f{}{}'.format(j,i) for i,j in product(*[range(4)]*2)])
R2.inject_variables()
R2_vars = np.array(R2.gens()).reshape((-1,4))
R2_vars

Defining f00, f10, f20, f30, f01, f11, f21, f31, f02, f12, f22, f32, f03, f13, f23, f33


array([[f00, f10, f20, f30],
       [f01, f11, f21, f31],
       [f02, f12, f22, f32],
       [f03, f13, f23, f33]], dtype=object)

In [7]:
#rewrite the 1D spline in terms of ymy1
py = 0
for mon in [f0, f1, f2, f3]:
    py += p.monomial_coefficient(mon)(ymy1, 0, 0)*mon
py

(-1/2*ymy1^3 + ymy1^2 - 1/2*ymy1)*f0 + (3/2*ymy1^3 - 5/2*ymy1^2 + 1)*f1 + (-3/2*ymy1^3 + 2*ymy1^2 + 1/2*ymy1)*f2 + (1/2*ymy1^3 - 1/2*ymy1^2)*f3

In [8]:
#evaluate the 1D interpolation 4 times in the x direction, each time for a different y value
b0 = p(*R2_vars[0])
b1 = p(*R2_vars[1])
b2 = p(*R2_vars[2])
b3 = p(*R2_vars[3])

# now interpolate these interpolations in the y direction
p2 = py(f0=b0, f1=b1, f2=b2, f3=b3)

# the resulting polynomial is quite big already.
# again, sage already understands this as a linear combination
# of the 16 function values, with polynomials as coefficients.
# we can print each of those polynomials separately
for mon in R2.gens():
    print(str(mon) + ":" + str(p2.monomial_coefficient(mon)).replace("^", "**"))

f00:1/4*xmx1**3*ymy1**3 - 1/2*xmx1**3*ymy1**2 - 1/2*xmx1**2*ymy1**3 + 1/4*xmx1**3*ymy1 + xmx1**2*ymy1**2 + 1/4*xmx1*ymy1**3 - 1/2*xmx1**2*ymy1 - 1/2*xmx1*ymy1**2 + 1/4*xmx1*ymy1
f10:-3/4*xmx1**3*ymy1**3 + 3/2*xmx1**3*ymy1**2 + 5/4*xmx1**2*ymy1**3 - 3/4*xmx1**3*ymy1 - 5/2*xmx1**2*ymy1**2 + 5/4*xmx1**2*ymy1 - 1/2*ymy1**3 + ymy1**2 - 1/2*ymy1
f20:3/4*xmx1**3*ymy1**3 - 3/2*xmx1**3*ymy1**2 - xmx1**2*ymy1**3 + 3/4*xmx1**3*ymy1 + 2*xmx1**2*ymy1**2 - 1/4*xmx1*ymy1**3 - xmx1**2*ymy1 + 1/2*xmx1*ymy1**2 - 1/4*xmx1*ymy1
f30:-1/4*xmx1**3*ymy1**3 + 1/2*xmx1**3*ymy1**2 + 1/4*xmx1**2*ymy1**3 - 1/4*xmx1**3*ymy1 - 1/2*xmx1**2*ymy1**2 + 1/4*xmx1**2*ymy1
f01:-3/4*xmx1**3*ymy1**3 + 5/4*xmx1**3*ymy1**2 + 3/2*xmx1**2*ymy1**3 - 5/2*xmx1**2*ymy1**2 - 3/4*xmx1*ymy1**3 - 1/2*xmx1**3 + 5/4*xmx1*ymy1**2 + xmx1**2 - 1/2*xmx1
f11:9/4*xmx1**3*ymy1**3 - 15/4*xmx1**3*ymy1**2 - 15/4*xmx1**2*ymy1**3 + 25/4*xmx1**2*ymy1**2 + 3/2*xmx1**3 + 3/2*ymy1**3 - 5/2*xmx1**2 - 5/2*ymy1**2 + 1
f21:-9/4*xmx1**3*ymy1**3 + 15/4*xmx1**3*

These polynomials can be hardcoded in the cuda kernels, but it is more convinient to store there coefficients in a file.

In [9]:
pos_mons = [xmx1^i*ymy1^j for i,j in product(*[range(4)]*2)]
M = Matrix([[p2.monomial_coefficient(val_mon).monomial_coefficient(pos_mon) for pos_mon in pos_mons] for val_mon in R2.gens()])
coeff_string = str(list(np.array(M).ravel()))[1:-1]
coeff_string

'0.0, 0.0, 0.0, 0.0, 0.0, 0.25, -0.5, 0.25, 0.0, -0.5, 1.0, -0.5, 0.0, 0.25, -0.5, 0.25, 0.0, -0.5, 1.0, -0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 1.25, -2.5, 1.25, 0.0, -0.75, 1.5, -0.75, 0.0, 0.0, 0.0, 0.0, 0.0, -0.25, 0.5, -0.25, 0.0, -1.0, 2.0, -1.0, 0.0, 0.75, -1.5, 0.75, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, -0.5, 0.25, 0.0, -0.25, 0.5, -0.25, 0.0, 0.0, 0.0, 0.0, -0.5, 0.0, 1.25, -0.75, 1.0, 0.0, -2.5, 1.5, -0.5, 0.0, 1.25, -0.75, 1.0, 0.0, -2.5, 1.5, 0.0, 0.0, 0.0, 0.0, -2.5, 0.0, 6.25, -3.75, 1.5, 0.0, -3.75, 2.25, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, -1.25, 0.75, 2.0, 0.0, -5.0, 3.0, -1.5, 0.0, 3.75, -2.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5, 0.0, 1.25, -0.75, 0.5, 0.0, -1.25, 0.75, 0.0, 0.0, 0.0, 0.0, 0.0, -0.25, -1.0, 0.75, 0.0, 0.5, 2.0, -1.5, 0.0, -0.25, -1.0, 0.75, 0.0, 0.5, 2.0, -1.5, 0.0, 0.0, 0.0, 0.0, 0.0, -1.25, -5.0, 3.75, 0.0, 0.75, 3.0, -2.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 1.0, -0.75, 0.0, 1.0, 4.0, -3.0, 0.0, -0.75, -3.0, 2.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [10]:
with open("cubic_2D_coefficients.inc", "w") as file:
    file.write(coeff_string)

3D cubic spline coefficients
============================

In [11]:
R3 = PolynomialRing(S, ['f{}{}{}'.format(k,j,i) for i,j,k in product(*[range(4)]*3)])
R3.inject_variables()
R3_vars = np.array(R3.gens()).reshape((4,-1))
R3_vars

Defining f000, f100, f200, f300, f010, f110, f210, f310, f020, f120, f220, f320, f030, f130, f230, f330, f001, f101, f201, f301, f011, f111, f211, f311, f021, f121, f221, f321, f031, f131, f231, f331, f002, f102, f202, f302, f012, f112, f212, f312, f022, f122, f222, f322, f032, f132, f232, f332, f003, f103, f203, f303, f013, f113, f213, f313, f023, f123, f223, f323, f033, f133, f233, f333


array([[f000, f100, f200, f300, f010, f110, f210, f310, f020, f120, f220,
        f320, f030, f130, f230, f330],
       [f001, f101, f201, f301, f011, f111, f211, f311, f021, f121, f221,
        f321, f031, f131, f231, f331],
       [f002, f102, f202, f302, f012, f112, f212, f312, f022, f122, f222,
        f322, f032, f132, f232, f332],
       [f003, f103, f203, f303, f013, f113, f213, f313, f023, f123, f223,
        f323, f033, f133, f233, f333]], dtype=object)

In [12]:
pz = 0
for mon in [f0, f1, f2, f3]:
    pz += p.monomial_coefficient(mon)(zmz1, 0, 0)*mon
pz

(-1/2*zmz1^3 + zmz1^2 - 1/2*zmz1)*f0 + (3/2*zmz1^3 - 5/2*zmz1^2 + 1)*f1 + (-3/2*zmz1^3 + 2*zmz1^2 + 1/2*zmz1)*f2 + (1/2*zmz1^3 - 1/2*zmz1^2)*f3

In [13]:
c0 = p2(*R3_vars[0])
c1 = p2(*R3_vars[1])
c2 = p2(*R3_vars[2])
c3 = p2(*R3_vars[3])
p3 = pz(f0=c0, f1=c1, f2=c2, f3=c3)
for mon in R3.gens():
    print(str(mon) + ":" + str(p3.monomial_coefficient(mon)))

f000:-1/8*xmx1^3*ymy1^3*zmz1^3 + 1/4*xmx1^3*ymy1^3*zmz1^2 + 1/4*xmx1^3*ymy1^2*zmz1^3 + 1/4*xmx1^2*ymy1^3*zmz1^3 - 1/8*xmx1^3*ymy1^3*zmz1 - 1/2*xmx1^3*ymy1^2*zmz1^2 - 1/2*xmx1^2*ymy1^3*zmz1^2 - 1/8*xmx1^3*ymy1*zmz1^3 - 1/2*xmx1^2*ymy1^2*zmz1^3 - 1/8*xmx1*ymy1^3*zmz1^3 + 1/4*xmx1^3*ymy1^2*zmz1 + 1/4*xmx1^2*ymy1^3*zmz1 + 1/4*xmx1^3*ymy1*zmz1^2 + xmx1^2*ymy1^2*zmz1^2 + 1/4*xmx1*ymy1^3*zmz1^2 + 1/4*xmx1^2*ymy1*zmz1^3 + 1/4*xmx1*ymy1^2*zmz1^3 - 1/8*xmx1^3*ymy1*zmz1 - 1/2*xmx1^2*ymy1^2*zmz1 - 1/8*xmx1*ymy1^3*zmz1 - 1/2*xmx1^2*ymy1*zmz1^2 - 1/2*xmx1*ymy1^2*zmz1^2 - 1/8*xmx1*ymy1*zmz1^3 + 1/4*xmx1^2*ymy1*zmz1 + 1/4*xmx1*ymy1^2*zmz1 + 1/4*xmx1*ymy1*zmz1^2 - 1/8*xmx1*ymy1*zmz1
f100:3/8*xmx1^3*ymy1^3*zmz1^3 - 3/4*xmx1^3*ymy1^3*zmz1^2 - 3/4*xmx1^3*ymy1^2*zmz1^3 - 5/8*xmx1^2*ymy1^3*zmz1^3 + 3/8*xmx1^3*ymy1^3*zmz1 + 3/2*xmx1^3*ymy1^2*zmz1^2 + 5/4*xmx1^2*ymy1^3*zmz1^2 + 3/8*xmx1^3*ymy1*zmz1^3 + 5/4*xmx1^2*ymy1^2*zmz1^3 - 3/4*xmx1^3*ymy1^2*zmz1 - 5/8*xmx1^2*ymy1^3*zmz1 - 3/4*xmx1^3*ymy1*zmz1^2 - 5/2*xm

### A more compact data representation with matrix

In [14]:
pos_mons = [xmx1^i*ymy1^j*zmz1^k for i,j,k in product(*[range(4)]*3)]
M = Matrix([[p3.monomial_coefficient(val_mon).monomial_coefficient(pos_mon) for pos_mon in pos_mons] for val_mon in R3.gens()])
coeff_string = str(list(np.array(M).ravel()))[1:-1]
coeff_string

'0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.125, 0.25, -0.125, 0.0, 0.25, -0.5, 0.25, 0.0, -0.125, 0.25, -0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, -0.5, 0.25, 0.0, -0.5, 1.0, -0.5, 0.0, 0.25, -0.5, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, -0.125, 0.25, -0.125, 0.0, 0.25, -0.5, 0.25, 0.0, -0.125, 0.25, -0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, -0.5, 0.25, 0.0, -0.5, 1.0, -0.5, 0.0, 0.25, -0.5, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.625, 1.25, -0.625, 0.0, 1.25, -2.5, 1.25, 0.0, -0.625, 1.25, -0.625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.375, -0.75, 0.375, 0.0, -0.75, 1.5, -0.75, 0.0, 0.375, -0.75, 0.375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, -0.25, 0.125, 0.0, -0.25, 0.5, -0.25, 0.0, 0.125, -0.25, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, -1.0, 0.5, 0.0, -1.0, 2.0, -1.0, 0.0, 0.5, -1.0, 0.5, 0.0, 0.0, 0.0, 0

In [15]:
with open("cubic_3D_coefficients.inc", "w") as file:
    file.write(coeff_string)